<a href="https://colab.research.google.com/github/junya17/RAG-Question-Answering-Gpt-Bert/blob/main/RAG_Question_Answering_Gpt_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu

In [ ]:
import torch
import numpy as np
import faiss
from transformers import GPT2LMHeadModel, GPT2Tokenizer, BertTokenizer, BertModel

# GPT-2モデルとトークナイザーのロード
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")

# BERTモデルとトークナイザーのロード
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_sentence_vector(sentence, model, tokenizer):
    """文のBERTベクトルを取得する関数"""
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, max_length=128)
    outputs = model(**inputs)
    sentence_vector = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return sentence_vector.reshape(-1)

def search_closest_question(query, index, qa_pairs, model, tokenizer):
    """FAISSインデックスを使用して、質問に最も近いQAペアを検索する関数"""
    query_vector = get_sentence_vector(query, model, tokenizer)
    _, I = index.search(np.array([query_vector]), k=1)
    return qa_pairs[I[0][0]]

def generate_answer_with_gpt(document, query, tokenizer, model, max_length=300, temperature=1.0, top_k=50, top_p=0.95):
    tokenizer.pad_token = tokenizer.eos_token

    combined_input = query + " " + document
    inputs = tokenizer.encode_plus(combined_input, return_tensors='pt', padding=True, truncation=True, max_length=128)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        do_sample=True  # この行を追加
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

def extract_answer(text, query):
    # 質問の後の部分を見つける
    answer_start = text.find(query) + len(query)
    if answer_start > -1:
        # 質問の後のテキストを抽出
        answer_text = text[answer_start:].strip()
        # 最初のピリオドまでのテキストを回答として返す
        answer_end = answer_text.find(".")
        if answer_end > -1:
            answer_text = answer_text[:answer_end]

        # # 不要な繰り返しを除去するための追加ロジック
        # # 例: 特定の繰り返しパターンを探して削除
        # repeat_pattern = "私は、その言うです。"
        # if repeat_pattern in answer_text:
        #     answer_text = answer_text.split(repeat_pattern)[0]

        return answer_text
    else:
        return "回答が見つかりませんでした。"

# QAペアとベクトルデータベースの準備
qa_pairs = [
    {
        "question": "東京の名所はどこですか？",
        "answer": "浅草寺、スカイツリー、渋谷の交差点",
        "documents": [
            "浅草寺は東京の有名な歴史的寺院です。",
            "東京スカイツリーは高さ634メートルのタワーで、展望台からの景色が素晴らしい。",
            "渋谷の交差点は、世界でも有名な繁忙な交差点です。"
        ]
    }
    # ここに追加のQAペアを追加可能
]

vectors = [get_sentence_vector(pair['question'], bert_model, bert_tokenizer) for pair in qa_pairs]
dim = vectors[0].shape[0]
index = faiss.IndexFlatL2(dim)
index.add(np.array(vectors))

# メイン処理
user_question = "東京の名所はどこですか？"
closest_pair = search_closest_question(user_question, index, qa_pairs, bert_model, bert_tokenizer)
document_text = closest_pair["answer"]
generated_answer = generate_answer_with_gpt(document_text, user_question, gpt_tokenizer, gpt_model)
final_answer = extract_answer(generated_answer, user_question)

print("Final answer:", final_answer)
